In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = "../"

import time
import os
import pandas as pd
from utils.pandas_extented_filters import filter_remove_null_nan_empty_entries
from utils.column_names_reader import get_col_names_from_gzip

In [2]:
# merging the downloaded dataset
def clean_df(data_filepath):
    df = pd.read_csv(data_filepath, delim_whitespace=False, sep="\t")
    df = df.drop_duplicates(subset="snp_id", keep="first")
    return df

data_filepath = home_dir+"data/dbsnp/search_results/dbsnp_HumanMissenseALFAVariants_0.txt"
df_0 = clean_df(data_filepath)

data_filepath = home_dir+"data/dbsnp/search_results/dbsnp_HumanMissenseALFAVariants_1.txt"
df_1 = clean_df(data_filepath)

data_filepath = home_dir+"data/dbsnp/search_results/dbsnp_HumanMissenseALFAVariants_2.txt"
df_2 = clean_df(data_filepath)

data_filepath = home_dir+"data/dbsnp/search_results/dbsnp_HumanMissenseALFAVariants_3.txt"
df_3 = clean_df(data_filepath)

In [3]:
# removing duplicate data points and variants with no protein variants
df = pd.concat([df_0, df_1, df_2, df_3], ignore_index=True)
df = df.drop_duplicates(subset="snp_id", keep="first")
print(df.shape)
df = filter_remove_null_nan_empty_entries(df, a_col_name="variations") # NNE: null, nan or empty variants
print(df.shape)
print(df.columns)

(5140350, 14)
Number of NAN rows removed: 151569
Number of NULL rows removed: 0
Number of empty rows removed: 0
(4988781, 14)
Index(['snp_id', 'acc', 'chrpos', 'spdi', 'tax_id', 'snp_class', 'create_date',
       'update_date', 'clinical_significance', 'fxn_class', 'validated',
       'genes', 'mafs', 'variations'],
      dtype='object')


In [4]:
# parsing gene-symbols
initially_cleaned_df = df.copy()
initially_cleaned_df["snp_id"] = initially_cleaned_df["snp_id"].apply(lambda x: "rs"+str(x))
dbsnp_ids = initially_cleaned_df["snp_id"].unique().tolist()
print(len(dbsnp_ids))
initially_cleaned_df[["genes"]].value_counts()

4988781


genes                                                                  
TTN:7273,TTN-AS1:100506866                                                 9381
MUC16:94025                                                                8341
MUC4:4585                                                                  5782
TTN:7273                                                                   5151
AHNAK2:113146                                                              4726
                                                                           ... 
CASP5:838,LOC124902742:124902742                                              1
SLX1B:79008,BOLA2:552900,SLX1B-SULT1A4:100526831,BOLA2-SMG1P6:107282092       1
RASSF7:8045,LMNTD2:256329,LMNTD2-AS1:692247                                   1
CASP4:837,LOC124902813:124902813                                              1
EDA:1896,MIR676:100500887                                                     1
Length: 27296, dtype: int64

In [9]:
initially_cleaned_df[initially_cleaned_df["snp_id"]=="rs1935"]["variations"].values

array(['NP_116165.1:p.Glu2535Asp,NP_116165.1:p.Glu2535Asp,NP_001305082.1:p.Glu2247Asp,NP_001305082.1:p.Glu2247Asp,NP_001269877.1:p.Glu2353Asp,NP_001269877.1:p.Glu2353Asp,NP_001309181.1:p.Glu2497Asp,NP_001309181.1:p.Glu2497Asp,NP_001309183.1:p.Glu2316Asp,NP_001309183.1:p.Glu2316Asp,NP_001305083.1:p.Glu2353Asp,NP_001305083.1:p.Glu2353Asp,NP_001309187.1:p.Glu2316Asp,NP_001309187.1:p.Glu2316Asp'],
      dtype=object)

In [5]:
initially_cleaned_df.head(20)

,snp_id,acc,chrpos,spdi,tax_id,snp_class,create_date,update_date,clinical_significance,fxn_class,validated,genes,mafs,variations
0,rs1491004261,NC_000002.12,2:128176888,NC_000002.12:128176887:T:C,9606,snv,2017/11/09 09:55,2022/10/12 21:08,NaN,"non_coding_transcript_variant,coding_sequence_...","by-frequency,by-alfa,by-cluster",UGGT1:56886,"GnomAD_exomes:C=0.000004/1,TOPMED:C=0.000008/2...",NP_064505.1:p.Ile1205Thr
1,rs1491004253,NC_000005.10,5:53042133,"NC_000005.10:53042132:G:A,NC_000005.10:5304213...",9606,snv,2017/11/09 09:55,2022/10/13 07:59,NaN,"coding_sequence_variant,non_coding_transcript_...","by-frequency,by-alfa,by-cluster",ITGA2:3673,"GnomAD_exomes:T=0.000004/1,TOPMED:A=0.000004/1...","NP_002194.2:p.Trp69Ter,NP_002194.2:p.Trp69Cys"
2,rs1491004226,NC_000020.11,20:33663605,"NC_000020.11:33663604:C:G,NC_000020.11:3366360...",9606,snv,2017/11/09 09:55,2022/10/13 17:00,NaN,"genic_upstream_transcript_variant,5_prime_UTR_...","by-frequency,by-alfa,by-cluster","NECAB3:63941,C20orf144:128864",ALFA:T=0./0,"NP_543015.1:p.Ala67Gly,NP_543015.1:p.Ala67Val"
3,rs1491003662,NC_000002.12,2:26946984,NC_000002.12:26946983:G:A,9606,snv,2017/11/09 09:55,2022/10/17 05:04,NaN,"coding_sequence_variant,missense_variant","by-frequency,by-alfa,by-cluster",DPYSL5:56896,"GnomAD:A=0.000007/1,TOPMED:A=0.000004/1,ALFA:A...","NP_064519.2:p.Gly562Ser,NP_001240653.1:p.Gly56..."
4,rs1491003598,NC_000001.11,1:205269218,NC_000001.11:205269217:A:G,9606,snv,2017/11/09 09:55,2022/10/12 18:08,NaN,"coding_sequence_variant,missense_variant","by-frequency,by-alfa,by-cluster",TMCC2:9911,"GnomAD_exomes:G=0.000004/1,TOMMO:G=0.000035/1,...","NP_055673.2:p.Lys339Arg,NP_001229854.1:p.Lys26..."
5,rs1491003486,NC_000001.11,1:9262263,NC_000001.11:9262262:G:A,9606,snv,2017/11/09 09:55,2022/10/12 16:11,NaN,"coding_sequence_variant,missense_variant,5_pri...","by-frequency,by-alfa,by-cluster",H6PD:9563,"GnomAD_exomes:A=0.000012/3,TOPMED:A=0.000004/1...","NP_004276.2:p.Ser317Asn,NP_001269516.1:p.Ser32..."
6,rs1491003469,NC_000001.11,1:210683706,NC_000001.11:210683705:C:G,9606,snv,2017/11/09 09:55,2022/10/17 04:45,NaN,"coding_sequence_variant,missense_variant","by-frequency,by-alfa,by-cluster",KCNH1:3756,"GnomAD:G=0.000036/5,GnomAD_exomes:G=0.000016/4...","NP_002229.1:p.Asp822His,NP_758872.1:p.Asp849His"
7,rs1491003245,NC_000002.12,2:105881549,NC_000002.12:105881548:C:T,9606,snv,2017/11/09 09:55,2022/10/12 20:41,NaN,"coding_sequence_variant,intron_variant,missens...","by-frequency,by-alfa,by-cluster",NCK2:8440,"GnomAD:T=0.000007/1,GnomAD_exomes:T=0.000004/1...","NP_003572.2:p.Arg150Trp,NP_001004720.1:p.Arg15..."
8,rs1491002990,NC_000006.12,6:122447250,"NC_000006.12:122447249:G:C,NC_000006.12:122447...",9606,snv,2017/11/09 09:55,2022/10/13 10:49,NaN,"coding_sequence_variant,missense_variant","by-frequency,by-alfa,by-cluster",SERINC1:57515,"GnomAD:C=0.000007/1,TOPMED:C=0.000008/2,ALFA:T...","NP_065806.1:p.Pro289Arg,NP_065806.1:p.Pro289Gln"
9,rs1491002863,NC_000009.12,9:2047295,"NC_000009.12:2047294:G:A,NC_000009.12:2047294:G:C",9606,snv,2017/11/09 09:55,2022/10/16 10:50,NaN,"missense_variant,genic_upstream_transcript_var...","by-frequency,by-alfa,by-cluster",SMARCA2:6595,"GnomAD:C=0./0,ALFA:C=0./0","NP_003061.3:p.Arg286Gln,NP_003061.3:p.Arg286Pr..."


### Reading and filtering ALFA data as it is big

In [25]:
# do not run this unless you changed something before, it takes 1 hour
# better use the following block
from utils.column_names_reader import get_col_names_from_gzip
filepath = home_dir+"data/ALFA_population_freq/freq.vcf.gz"
col_names = get_col_names_from_gzip(filepath, "#CHROM")
alfa_df_iterator = pd.read_csv(filepath, compression='gzip', chunksize=1000000, comment="#", names=col_names, delim_whitespace=False, sep="\t")
# alfa_df_iterator.__next__() # first df 

chunk_list = []
for i, alfa_df in enumerate(alfa_df_iterator):
    chunk = alfa_df[alfa_df["ID"].isin(dbsnp_ids)]
    chunk_list.append(chunk)
    print(i, chunk.shape)
    # if i==0: break

filtered_alfa_df = pd.concat(chunk_list)  
print(filtered_alfa_df.shape)
print(filtered_alfa_df.columns)
filtered_alfa_df

0 (19889, 21)
1 (5336, 21)
2 (9148, 21)
3 (13846, 21)
4 (10780, 21)
5 (11830, 21)
6 (13104, 21)
7 (9094, 21)
8 (13853, 21)
9 (6234, 21)
10 (10527, 21)
11 (9397, 21)
12 (11384, 21)
13 (14070, 21)
14 (9227, 21)
15 (2668, 21)
16 (11260, 21)
17 (2431, 21)
18 (2125, 21)
19 (5695, 21)
20 (3971, 21)
21 (1675, 21)
22 (3139, 21)
23 (3918, 21)
24 (361, 21)
25 (3480, 21)
26 (3361, 21)
27 (5891, 21)
28 (6887, 21)
29 (654, 21)
30 (4283, 21)
31 (1847, 21)
32 (711, 21)
33 (13125, 21)
34 (7633, 21)
35 (6052, 21)
36 (4362, 21)
37 (0, 21)
38 (0, 21)
39 (4314, 21)
40 (8112, 21)
41 (24622, 21)
42 (19463, 21)
43 (18059, 21)
44 (13846, 21)
45 (2027, 21)
46 (4562, 21)
47 (8178, 21)
48 (4510, 21)
49 (4567, 21)
50 (7363, 21)
51 (6500, 21)
52 (3521, 21)
53 (450, 21)
54 (1052, 21)
55 (5528, 21)
56 (9333, 21)
57 (12048, 21)
58 (9030, 21)
59 (4372, 21)
60 (4553, 21)
61 (3738, 21)
62 (3485, 21)
63 (7571, 21)
64 (12840, 21)
65 (6017, 21)
66 (5233, 21)
67 (7201, 21)
68 (3100, 21)
69 (2704, 21)
70 (9736, 21)
71 (5993,

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMN10492695,...,SAMN10492697,SAMN10492698,SAMN10492699,SAMN10492700,SAMN10492701,SAMN10492702,SAMN11605645,SAMN10492703,SAMN10492704,SAMN10492705
14405,NC_000001.11,69116,rs757299236,G,A,.,.,.,AN:AC,7618:0,...,84:0,2708:0,146:0,610:0,24:0,94:0,470:1,2816:0,108:0,11862:1
14410,NC_000001.11,69134,rs781394307,A,G,.,.,.,AN:AC,7618:9,...,84:0,2708:0,146:0,610:5,24:0,94:0,470:0,2816:0,108:0,11862:14
14414,NC_000001.11,69149,rs1458521218,T,A,.,.,.,AN:AC,7618:3,...,84:0,2708:0,146:1,610:0,24:0,94:0,470:0,2816:0,108:0,11862:4
14419,NC_000001.11,69184,rs1231643173,G,A,.,.,.,AN:AC,7618:0,...,84:0,2708:0,146:0,610:0,24:0,94:0,470:0,2816:0,108:0,11862:0
14422,NC_000001.11,69200,rs1453226491,T,C,.,.,.,AN:AC,7618:0,...,84:0,2708:5,146:0,610:0,24:0,94:0,470:0,2816:5,108:0,11862:5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904165833,NW_003571061.2,275107,rs772563100,T,C,.,.,.,AN:AC,5918:1,...,14:0,584:0,0:0,0:0,22:0,0:0,2218:0,592:0,36:0,8764:1
904165847,NW_003571061.2,275289,rs1469647256,A,G,.,.,.,AN:AC,16262:1,...,0:0,20:0,350:0,18:0,0:0,0:0,3936:0,20:0,0:0,20586:1
904165849,NW_003571061.2,275302,rs151150114,G,C,.,.,.,AN:AC,22180:4,...,14:0,602:0,350:0,18:0,22:0,0:0,6154:0,610:0,36:0,29348:4
904165850,NW_003571061.2,275304,rs1334679620,A,G,.,.,.,AN:AC,16262:1,...,0:0,20:0,350:0,18:0,0:0,0:0,3936:0,20:0,0:0,20586:1


In [6]:
filtered_alpha_filepath = home_dir+"data/temp_popu_freq_data/alfa_filtered.tsv"
# filtered_alfa_df.to_csv(filtered_alpha_filepath, index=False, sep="\t")
filtered_alfa_df = pd.read_csv(filtered_alpha_filepath, sep="\t")
print(filtered_alfa_df.columns)
print(filtered_alfa_df.shape)
print(filtered_alfa_df["ID"].unique().shape)

Index(['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT',
       'SAMN10492695', 'SAMN10492696', 'SAMN10492697', 'SAMN10492698',
       'SAMN10492699', 'SAMN10492700', 'SAMN10492701', 'SAMN10492702',
       'SAMN11605645', 'SAMN10492703', 'SAMN10492704', 'SAMN10492705'],
      dtype='object')
(4988568, 21)
(4988568,)


In [7]:
filtered_alfa_df[filtered_alfa_df["ID"]=="rs200348921"]


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMN10492695,...,SAMN10492697,SAMN10492698,SAMN10492699,SAMN10492700,SAMN10492701,SAMN10492702,SAMN11605645,SAMN10492703,SAMN10492704,SAMN10492705
1497138,NC_000005.10,141374104,rs200348921,T,"C,A,G",.,.,.,AN:AC,"20198:13,1,2",...,"112:0,0,0","3368:0,0,0","146:0,0,0","610:0,0,0","56:0,0,0","98:0,0,0","2780:9,0,0","3490:0,0,0","168:0,0,0","27490:22,1,2"


In [29]:
filtered_alfa_df["REF"].value_counts()

C                           1664374
G                           1652765
A                            836569
T                            834852
TA                                1
CCACTTCTTTAATCACCACCACCA          1
CCAACCT                           1
CCTT                              1
ATAGCCTCTCCTCCTGTTCACGT           1
GCCT                              1
AT                                1
ATCT                              1
Name: REF, dtype: int64

### Mapping to MANE proteins

In [31]:
# loading MANE data
mane_df = pd.read_csv(home_dir+"data/refseq/MANE.GRCh38.v1.0.summary.txt.gz", compression='gzip', delim_whitespace=False, sep="\t") # MANE refseq-protein mapping
print(mane_df.shape)
print(mane_df.columns)

print(mane_df["MANE_status"].value_counts())
print(mane_df["symbol"].value_counts()) # some gene can be mapped to multiple proteins because of MANE select and Plus clinical.

mane_short_df = mane_df[["symbol", "RefSeq_prot", "MANE_status"]]
# mane_short_df.value_counts()
mane_df[mane_df["symbol"]=="DYSF"]

(19120, 14)
Index(['#NCBI_GeneID', 'Ensembl_Gene', 'HGNC_ID', 'symbol', 'name',
       'RefSeq_nuc', 'RefSeq_prot', 'Ensembl_nuc', 'Ensembl_prot',
       'MANE_status', 'GRCh38_chr', 'chr_start', 'chr_end', 'chr_strand'],
      dtype='object')
MANE Select           19062
MANE Plus Clinical       58
Name: MANE_status, dtype: int64
CACNA1D         2
CACNA1C         2
DYNC2H1         2
SLC39A14        2
CLPB            2
               ..
ZNF432          1
SAFB2           1
DZIP3           1
MARF1           1
LOC122539214    1
Name: symbol, Length: 19062, dtype: int64


,#NCBI_GeneID,Ensembl_Gene,HGNC_ID,symbol,name,RefSeq_nuc,RefSeq_prot,Ensembl_nuc,Ensembl_prot,MANE_status,GRCh38_chr,chr_start,chr_end,chr_strand
5407,GeneID:8291,ENSG00000135636.16,HGNC:3097,DYSF,dysferlin,NM_003494.4,NP_003485.1,ENST00000258104.8,ENSP00000258104.3,MANE Plus Clinical,2,71453561,71686763,+
5408,GeneID:8291,ENSG00000135636.16,HGNC:3097,DYSF,dysferlin,NM_001130987.2,NP_001124459.1,ENST00000410020.8,ENSP00000386881.3,MANE Select,2,71466699,71686763,+


In [ ]:
# some dbSNP-ids are annotated to multiple genes. ie rs756964411 is mapped to PSMB7, NEK6, LOC100129034. 
# we map gene-symbols to the MANE gene-symbols, refseq proteins.
# this block takes >1 hour to run, better use the saved data from the next block
data = []
for i, row in enumerate(initially_cleaned_df.itertuples(index=False)):
    gene_symbols_list = [gene.split(":")[0] for gene in row.genes.split(",")]
    mane_things = mane_short_df[mane_short_df["symbol"].isin(gene_symbols_list)]
    mane_genes = mane_things["symbol"].unique().tolist()
    mane_proteins = mane_things["RefSeq_prot"].unique().tolist()

    variations = row.variations.split(",")
    mane_protein_variants = []
    for variation in variations:
        prot = variation.split(":")[0]
        if prot in mane_proteins:
            mane_protein_variants.append(variation)

    datam = row._asdict()
    datam["mane_genes"] = ",".join(mane_genes)
    datam["mane_proteins"] = ",".join(mane_proteins)
    datam["mane_protein_variants"] = ",".join(mane_protein_variants)

    data.append(datam)
    
    if i%100000==0: print(i)
    # if i==5: break

variants_with_mane_df = pd.DataFrame(data) 
print(variants_with_mane_df.shape)
print(variants_with_mane_df.columns)
print(variants_with_mane_df["snp_id"].unique().shape)
variants_with_mane_df

In [36]:
filepath = home_dir+"data/temp_popu_freq_data/variants_with_mane.tsv"
# variants_with_mane_df.to_csv(filepath, index=False, sep="\t")
variants_with_mane_df = pd.read_csv(filepath, sep="\t")
print(variants_with_mane_df.shape)
print(variants_with_mane_df.columns)
print(variants_with_mane_df["snp_id"].unique().shape)
variants_with_mane_df

(4988781, 17)
Index(['snp_id', 'acc', 'chrpos', 'spdi', 'tax_id', 'snp_class', 'create_date',
       'update_date', 'clinical_significance', 'fxn_class', 'validated',
       'genes', 'mafs', 'variations', 'mane_genes', 'mane_proteins',
       'mane_protein_variants'],
      dtype='object')
(4988781,)


,snp_id,acc,chrpos,spdi,tax_id,snp_class,create_date,update_date,clinical_significance,fxn_class,validated,genes,mafs,variations,mane_genes,mane_proteins,mane_protein_variants
0,rs1491004261,NC_000002.12,2:128176888,NC_000002.12:128176887:T:C,9606,snv,2017/11/09 09:55,2022/10/12 21:08,NaN,"non_coding_transcript_variant,coding_sequence_...","by-frequency,by-alfa,by-cluster",UGGT1:56886,"GnomAD_exomes:C=0.000004/1,TOPMED:C=0.000008/2...",NP_064505.1:p.Ile1205Thr,UGGT1,NP_064505.1,NP_064505.1:p.Ile1205Thr
1,rs1491004253,NC_000005.10,5:53042133,"NC_000005.10:53042132:G:A,NC_000005.10:5304213...",9606,snv,2017/11/09 09:55,2022/10/13 07:59,NaN,"coding_sequence_variant,non_coding_transcript_...","by-frequency,by-alfa,by-cluster",ITGA2:3673,"GnomAD_exomes:T=0.000004/1,TOPMED:A=0.000004/1...","NP_002194.2:p.Trp69Ter,NP_002194.2:p.Trp69Cys",ITGA2,NP_002194.2,"NP_002194.2:p.Trp69Ter,NP_002194.2:p.Trp69Cys"
2,rs1491004226,NC_000020.11,20:33663605,"NC_000020.11:33663604:C:G,NC_000020.11:3366360...",9606,snv,2017/11/09 09:55,2022/10/13 17:00,NaN,"genic_upstream_transcript_variant,5_prime_UTR_...","by-frequency,by-alfa,by-cluster","NECAB3:63941,C20orf144:128864",ALFA:T=0./0,"NP_543015.1:p.Ala67Gly,NP_543015.1:p.Ala67Val","NECAB3,C20orf144","NP_112509.3,NP_543015.1","NP_543015.1:p.Ala67Gly,NP_543015.1:p.Ala67Val"
3,rs1491003662,NC_000002.12,2:26946984,NC_000002.12:26946983:G:A,9606,snv,2017/11/09 09:55,2022/10/17 05:04,NaN,"coding_sequence_variant,missense_variant","by-frequency,by-alfa,by-cluster",DPYSL5:56896,"GnomAD:A=0.000007/1,TOPMED:A=0.000004/1,ALFA:A...","NP_064519.2:p.Gly562Ser,NP_001240653.1:p.Gly56...",DPYSL5,NP_064519.2,NP_064519.2:p.Gly562Ser
4,rs1491003598,NC_000001.11,1:205269218,NC_000001.11:205269217:A:G,9606,snv,2017/11/09 09:55,2022/10/12 18:08,NaN,"coding_sequence_variant,missense_variant","by-frequency,by-alfa,by-cluster",TMCC2:9911,"GnomAD_exomes:G=0.000004/1,TOMMO:G=0.000035/1,...","NP_055673.2:p.Lys339Arg,NP_001229854.1:p.Lys26...",TMCC2,NP_055673.2,NP_055673.2:p.Lys339Arg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4988776,rs334,NC_000011.10,11:5227002,"NC_000011.10:5227001:T:A,NC_000011.10:5227001:...",9606,snv,2000/09/19 17:02,2022/10/16 14:17,"protective,other,pathogenic,benign,likely-beni...","missense_variant,coding_sequence_variant","by-frequency,by-alfa,by-cluster",HBB:3043,"1000Genomes:A=0.028732/144,ExAC:A=0.004384/532...","NP_000509.1:p.Glu7Val,NP_000509.1:p.Glu7Gly,NP...",HBB,NP_000509.1,"NP_000509.1:p.Glu7Val,NP_000509.1:p.Glu7Gly,NP..."
4988777,rs300,NC_000008.11,8:19959376,NC_000008.11:19959375:A:G,9606,snv,2000/09/19 17:02,2022/10/14 11:06,"benign,benign-likely-benign","missense_variant,coding_sequence_variant","by-frequency,by-alfa,by-cluster",LPL:4023,"1000Genomes:G=0.003592/18,ExAC:G=0.001084/131,...",NP_000228.1:p.Thr379Ala,LPL,NP_000228.1,NP_000228.1:p.Thr379Ala
4988778,rs298,NC_000008.11,8:19959349,NC_000008.11:19959348:G:A,9606,snv,2000/09/19 17:02,2022/10/14 11:06,NaN,"missense_variant,coding_sequence_variant","by-frequency,by-alfa,by-cluster",LPL:4023,"ExAC:A=0.00005/6,GnomAD:A=0.000093/13,GnomAD_e...",NP_000228.1:p.Val370Met,LPL,NP_000228.1,NP_000228.1:p.Val370Met
4988779,rs268,NC_000008.11,8:19956018,NC_000008.11:19956017:A:G,9606,snv,2000/09/19 17:02,2022/10/14 11:06,"conflicting-interpretations-of-pathogenicity,u...","missense_variant,coding_sequence_variant","by-frequency,by-alfa,by-cluster",LPL:4023,"1000Genomes:G=0.004685/23,ALSPAC:G=0.018682/72...",NP_000228.1:p.Asn318Ser,LPL,NP_000228.1,NP_000228.1:p.Asn318Ser


In [40]:
variants_with_refseq_prot_variants_df = variants_with_mane_df.copy()
variants_with_refseq_prot_variants_df = filter_remove_null_nan_empty_entries(variants_with_refseq_prot_variants_df, a_col_name="mane_protein_variants") # NNE: null, nan or empty variants
print(variants_with_refseq_prot_variants_df.shape)
print(variants_with_refseq_prot_variants_df["snp_id"].unique().shape[0])
print(variants_with_refseq_prot_variants_df.columns)

Number of NAN rows removed: 174080
Number of NULL rows removed: 0
Number of empty rows removed: 0
(4814701, 17)
4814701
Index(['snp_id', 'acc', 'chrpos', 'spdi', 'tax_id', 'snp_class', 'create_date',
       'update_date', 'clinical_significance', 'fxn_class', 'validated',
       'genes', 'mafs', 'variations', 'mane_genes', 'mane_proteins',
       'mane_protein_variants'],
      dtype='object')


### Mapping to ALFA population freq

In [41]:
variants_with_popu_count_df = pd.merge(left=variants_with_refseq_prot_variants_df, right=filtered_alfa_df, left_on="snp_id", right_on="ID", how="inner")
print(variants_with_popu_count_df.shape)
print(variants_with_popu_count_df["snp_id"].unique().shape)

(4814631, 38)
(4814631,)


In [42]:
variants_with_popu_count_df.to_csv(home_dir+"data/temp_popu_freq_data/snps_with_mane_and_popu_count_raw.tsv", index=False, sep="\t")
# this is ~3gb, so saving a short form by selecting cols

In [ ]:
cols_dict = {"snp_id":"snp_id", "CHROM":"chrom_acc_version", "POS":"chrom_pos_1indexed", "REF":"ref_allele", "ALT":"alt_alleles", 
             "mane_genes":"mane_genes", "mane_proteins":"mane_proteins", "mane_protein_variants":"mane_protein_variants", 
             "SAMN10492705":"alfa_popu_counts"}

variants_selected_cols_df = variants_with_popu_count_df[cols_dict.keys()].copy()
variants_selected_cols_df.rename(columns=cols_dict, inplace=True)
variants_selected_cols_df.to_csv(home_dir+"data/temp_popu_freq_data/snps_with_mane_and_popu_count_raw_short.tsv", index=False, sep="\t")